<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/baseball_kaggle_comps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pybaseball
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.8/418.8 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 KB 18.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pybaseball as bb

In [ ]:
df = bb.statcast(start_dt='2022-03-31', end_dt='2022-10-06')
df['is_top'] = 0
df.loc[df['inning_topbot']=='Top', 'is_top'] = 1
df['uid'] = (df['game_pk'].astype(str).add(df['inning'].astype('str')).add(df['is_top'].astype('str')).add(df['outs_when_up'].astype('str')).add(df['balls'].astype('str')).add(df['strikes'].astype('str')).add(df['pitch_number'].astype('str'))).astype(int)

# used for xstrike dataset
df['is_strike'] = 0
df.loc[df['description']=='called_strike', 'is_strike'] = 1

# used for xhr dataset
df['is_hr'] = 0
df.loc[df['events']=='home_run', 'is_hr'] = 1

df = df.reset_index(drop=True)
df.shape

This is a large query, it may take a moment to complete


/usr/local/lib/python3.8/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 190/190 [04:35<00:00,  1.45s/it]


(725736, 92)

In [ ]:
df.columns.values

array(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year',
       'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire',
       'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle',
       'effective_speed', 'release_spin_rate', 'release_extension',
       'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4',
       'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
       'release_pos_y', 'estima

In [ ]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,CH,2022-10-05,80.8,-0.76,6.61,"Baker, Bryan",624415,641329,field_out,hit_into_play,...,5,4,5,4,5,Infield shift,Standard,213,0.035,-0.099
1,FF,2022-10-05,97.7,-0.58,6.6,"Baker, Bryan",643376,641329,strikeout,swinging_strike,...,5,4,5,4,5,Standard,Standard,189,0.052,-0.134
2,CH,2022-10-05,84.9,-0.55,6.58,"Baker, Bryan",643376,641329,NaN,ball,...,5,4,5,4,5,Standard,Standard,212,0.0,0.023
3,FF,2022-10-05,97.2,-0.42,6.6,"Baker, Bryan",643376,641329,NaN,swinging_strike,...,5,4,5,4,5,Standard,Standard,192,0.0,-0.036
4,FC,2022-10-05,86.2,-0.55,6.64,"Baker, Bryan",643376,641329,NaN,called_strike,...,5,4,5,4,5,Standard,Standard,127,0.0,-0.031


# xstrikes comp

In [ ]:
id_feat = ['uid']
feats = ['sz_top', 'sz_bot','pitch_type','release_pos_x','release_pos_y',
         'release_pos_z','stand', 'p_throws','inning',
         'inning_topbot', 'outs_when_up', 'balls', 'strikes',
         'if_fielding_alignment','of_fielding_alignment','on_3b',
         'on_2b', 'on_1b','release_speed','spin_axis', 'release_spin_rate',
         'pfx_x','pfx_z', 'plate_x', 'plate_z']
target = 'is_strike'

df_xstrikes = df.loc[df['description'].isin(['ball','called_strike']), id_feat+feats+[target]].drop_duplicates(subset=['uid'])

for col in ['on_3b', 'on_2b', 'on_1b']:
  df_xstrikes[col] = df_xstrikes[col].fillna(0).astype(bool)

df_xstrikes = df_xstrikes.dropna()
df_xstrikes['uid'] = df_xstrikes['uid'].astype('category').cat.codes
x = df_xstrikes.dtypes
cols = x[x=='Float64'].index.values
for col in cols:
  df_xstrikes[col] = df_xstrikes[col].astype(float)

cols = x[x=='Int64'].index.values
for col in cols:
  df_xstrikes[col] = df_xstrikes[col].astype(int)

df_xstrikes = df_xstrikes.sort_values('uid').reset_index(drop=True)
df_xstrikes.shape




<ipython-input-111-3cb2193a9c94>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xstrikes['uid'] = df_xstrikes['uid'].astype('category').cat.codes
<ipython-input-111-3cb2193a9c94>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xstrikes[col] = df_xstrikes[col].astype(float)
<ipython-input-111-3cb2193a9c94>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

(311271, 27)

In [ ]:
df_xstrikes.sample(20)

,uid,sz_top,sz_bot,pitch_type,release_pos_x,release_pos_y,release_pos_z,stand,p_throws,inning,...,on_2b,on_1b,release_speed,spin_axis,release_spin_rate,pfx_x,pfx_z,plate_x,plate_z,is_strike
57966,57966,3.47,1.68,SI,-1.21,54.03,5.51,R,R,7,...,False,False,92.6,214,2344,-1.36,1.33,-0.16,1.39,1
231402,231402,3.49,1.64,SI,-1.91,54.78,5.74,L,R,1,...,False,False,90.6,220,2016,-1.07,0.31,-1.89,2.96,0
133814,133814,3.45,1.76,FF,2.05,54.12,6.82,L,L,2,...,False,False,88.9,157,2186,0.66,1.67,0.15,2.84,1
74716,74716,3.78,1.82,SI,1.66,54.74,5.72,R,L,8,...,False,True,94.8,129,2197,1.40,0.83,-1.18,2.63,0
280443,280443,3.50,1.61,SI,-2.30,53.67,4.53,R,R,6,...,True,False,90.8,229,2241,-1.24,0.63,-1.49,3.41,0
114562,114562,3.43,1.59,FF,0.62,53.59,6.32,R,L,5,...,False,False,96.0,140,2061,0.90,1.49,-0.28,1.99,1
66597,66597,3.43,1.67,FC,-2.64,53.94,5.84,L,R,9,...,False,True,93.4,196,2585,-0.02,1.10,0.37,1.92,1
228591,228591,3.58,1.81,FF,-0.80,55.19,6.88,R,R,6,...,False,True,94.2,197,2125,-0.60,1.67,-1.91,2.44,0
110747,110747,3.37,1.68,SI,2.50,54.56,5.19,R,L,9,...,False,True,93.4,113,2058,1.16,0.17,1.18,3.11,0
45912,45912,3.16,1.55,SL,-2.63,54.03,5.97,R,R,2,...,True,True,85.2,45,2591,1.07,-0.06,0.68,1.34,0


In [ ]:
import catboost as cb
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

In [ ]:
test = df_xstrikes.sample(20000, random_state=69)
train = df_xstrikes.loc[~df_xstrikes.index.isin(test.index.values)]
train['uid'] = train['uid'].astype(str)
test['uid'] = test['uid'].astype(str)
train.to_csv('drive/My Drive/community-comps/xstrikes/train.csv', index=False)
test.to_csv('drive/My Drive/community-comps/xstrikes/test.csv', index=False)
test.loc[:, ['uid', 'is_strike']].to_csv('drive/My Drive/community-comps/xstrikes/solution.csv', index=False)

train['test'] = 0
test['test'] = 1
data = train.append(test,ignore_index=True)

cols = [
    'pitch_type', 'stand', 'p_throws', 'inning_topbot',
       'if_fielding_alignment', 'of_fielding_alignment'
]
for col in cols:
  data[f"{col}_code"] = data[col].astype('category').cat.codes

<ipython-input-114-3544c6988ee1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['uid'] = train['uid'].astype(str)
<ipython-input-114-3544c6988ee1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['test'] = 0


In [ ]:
id_feat = ['uid']
feats = ['sz_top', 'sz_bot', 'release_pos_x', 'release_pos_y',
       'release_pos_z', 'inning', 'outs_when_up', 'balls', 'strikes',
       'on_3b', 'on_2b', 'on_1b', 'release_speed', 'spin_axis',
       'release_spin_rate', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','pitch_type_code', 'stand_code',
       'p_throws_code', 'inning_topbot_code',
       'if_fielding_alignment_code', 'of_fielding_alignment_code']

target = 'is_strike'

train_data = data.loc[data['test']==0, id_feat+feats+[target]]
test_data = data.loc[data['test']==1, id_feat+feats+[target]]
xtrain = xgb.DMatrix(train_data.loc[:, feats], train_data[target])
xtest = xgb.DMatrix(test_data.loc[:, feats], test_data[target])

kf = KFold(5)
s1 = pd.Series()
s2 = pd.Series()
for train_idx, test_idx in kf.split(train_data):
  _train_data = train_data.iloc[train_idx]
  _test_data = train_data.iloc[test_idx]
  _xtrain = xgb.DMatrix(_train_data.loc[:, feats], _train_data[target])
  _xtest = xgb.DMatrix(_test_data.loc[:, feats], _test_data[target])

  p = {
      'objective':'binary:logistic'
  }
  model = xgb.train(p, _xtrain)
  _s = pd.Series(model.predict(_xtest), index=_test_data.index)
  s1 = s1.append(_s)

  model = cb.CatBoostClassifier(iterations=250,verbose=False)
  model.fit(_train_data.loc[:, feats], _train_data[target])
  _s = pd.DataFrame(model.predict_proba(_test_data.loc[:, feats]), index=_test_data.index)[1]
  s2 = s2.append(_s)

train_data['xgb'] = s1
train_data['cb'] = s2

model = MLPClassifier()
model.fit(train_data.loc[:, ['xgb', 'cb']], train_data[target])

p = {
    'objective':'binary:logistic'
}
xgbm = xgb.train(p, xtrain)
test_data['xgb'] = pd.Series(xgbm.predict(xtest), index=test_data.index)

cbm = cb.CatBoostClassifier(iterations=250,verbose=False)
cbm.fit(train_data.loc[:, feats], train_data[target])
test_data['cb'] = pd.DataFrame(cbm.predict_proba(test_data.loc[:, feats]), index=test_data.index)[1]

test_data[f"{target}_pred"] = pd.DataFrame(model.predict_proba(test_data.loc[:, ['xgb', 'cb']]), index=test_data.index)[1]
test_data['uid'] = test_data['uid'].astype(str)
test_data.loc[:, ['uid', 'is_strike_pred']].rename(columns={'is_strike_pred':'is_strike'}).to_csv('drive/My Drive/community-comps/xstrikes/sample_solution.csv', index=False)

<ipython-input-115-9dc096066288>:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s1 = pd.Series()
<ipython-input-115-9dc096066288>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s2 = pd.Series()


# xhr comp

In [ ]:
import catboost as cb
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

In [ ]:
id_feat = ['uid']
feats = ['home_team','sz_top','sz_bot','pitch_type','release_pos_x',
         'release_pos_y','release_pos_z','stand','p_throws','inning',
         'inning_topbot','outs_when_up', 'balls','strikes','pitch_number',
         'at_bat_number','if_fielding_alignment','of_fielding_alignment',
         'on_3b','on_2b', 'on_1b','release_speed','spin_axis',
         'release_spin_rate','pfx_x','pfx_z', 'plate_x', 'plate_z','hc_x',
         'hc_y','launch_speed', 'launch_angle']
target = 'is_hr'

df_xhr = df.loc[df['description'].isin(['hit_into_play','foul']), id_feat+feats+[target]].drop_duplicates(subset=['uid'])

for col in ['on_3b', 'on_2b', 'on_1b']:
  df_xhr[col] = df_xhr[col].fillna(0).astype(bool)

df_xhr = df_xhr.dropna()
df_xhr['uid'] = df_xhr['uid'].astype('category').cat.codes
x = df_xhr.dtypes
cols = x[x=='Float64'].index.values
for col in cols:
  df_xhr[col] = df_xhr[col].astype(float)

cols = x[x=='Int64'].index.values
for col in cols:
  df_xhr[col] = df_xhr[col].astype(int)

df_xhr = df_xhr.sort_values('uid').reset_index(drop=True)
print(df_xhr.shape)

df_xhr.sample(20)




(117309, 34)


<ipython-input-139-15a97fadd46e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xhr['uid'] = df_xhr['uid'].astype('category').cat.codes
<ipython-input-139-15a97fadd46e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xhr[col] = df_xhr[col].astype(float)
<ipython-input-139-15a97fadd46e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

,uid,home_team,sz_top,sz_bot,pitch_type,release_pos_x,release_pos_y,release_pos_z,stand,p_throws,...,release_spin_rate,pfx_x,pfx_z,plate_x,plate_z,hc_x,hc_y,launch_speed,launch_angle,is_hr
8077,8077,MIL,3.25,1.51,FF,-1.04,53.96,5.80,R,R,...,2357,-0.47,1.47,0.26,3.21,141.49,58.71,90.6,32,0
27266,27266,PHI,3.30,1.58,SL,2.55,55.12,6.30,R,L,...,2266,-1.59,-0.20,-0.59,1.75,156.64,93.25,81.4,27,0
49777,49777,SF,3.20,1.56,KC,-1.54,54.39,6.39,L,R,...,2855,0.27,-1.49,0.02,2.51,173.99,136.35,102.0,6,0
83346,83346,DET,3.40,1.65,FF,-2.42,53.56,5.30,R,R,...,2251,-0.80,1.36,0.05,2.16,133.20,26.23,108.0,33,0
92012,92012,COL,2.99,1.46,FC,-1.32,54.18,6.25,R,R,...,2263,0.21,0.51,1.15,1.33,111.57,169.40,90.3,-38,0
48937,48937,SF,3.46,1.59,SL,-1.50,54.06,4.70,L,R,...,2600,0.98,0.41,-0.50,3.09,143.59,154.70,78.3,-17,0
67376,67376,WSH,3.06,1.43,SL,-2.16,54.11,6.41,L,R,...,2306,0.38,0.26,-0.33,1.66,118.38,144.18,102.8,-15,0
111972,111972,LAA,3.53,1.65,FF,2.58,54.01,5.95,L,L,...,2363,0.81,1.53,-0.32,2.91,76.69,50.67,100.6,28,0
63708,63708,NYM,3.24,1.53,FF,-1.29,54.16,5.76,R,R,...,2446,-1.03,1.30,0.56,3.19,151.20,165.89,106.2,5,0
54953,54953,OAK,3.44,1.66,FC,-1.21,54.37,5.83,L,R,...,2003,0.28,0.63,0.70,2.19,201.62,130.97,95.4,5,0


In [ ]:

import pandas as pd
test = pd.read_csv('drive/My Drive/community-comps/xhr/test.csv')
test.shape

(20000, 34)

In [ ]:
test.drop('is_hr', axis=1).to_csv('drive/My Drive/community-comps/xhr/test.csv', index=False)

In [ ]:
test = df_xhr.sample(20000, random_state=420)
train = df_xhr.loc[~df_xhr.index.isin(test.index.values)]
train['uid'] = train['uid'].astype(str)
test['uid'] = test['uid'].astype(str)
train.to_csv('drive/My Drive/community-comps/xhr/train.csv', index=False)
test.to_csv('drive/My Drive/community-comps/xhr/test.csv', index=False)
test.loc[:, ['uid', 'is_hr']].to_csv('drive/My Drive/community-comps/xhr/solution.csv', index=False)

train['test'] = 0
test['test'] = 1
data = train.append(test,ignore_index=True)

cols = [
    'home_team', 'pitch_type', 'stand', 'p_throws',
    'inning_topbot', 'if_fielding_alignment', 'of_fielding_alignment'
]
for col in cols:
  data[f"{col}_code"] = data[col].astype('category').cat.codes

<ipython-input-143-f02682ac09b2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['uid'] = train['uid'].astype(str)
<ipython-input-143-f02682ac09b2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['test'] = 0


In [ ]:
id_feat = ['uid']
feats = ['sz_top', 'sz_bot', 'release_pos_x', 'release_pos_y',
       'release_pos_z', 'inning', 'outs_when_up', 'balls', 'strikes',
       'pitch_number', 'at_bat_number', 'on_3b', 'on_2b', 'on_1b',
       'release_speed', 'spin_axis', 'release_spin_rate', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'hc_x', 'hc_y', 'launch_speed',
       'launch_angle','home_team_code',
       'pitch_type_code', 'stand_code', 'p_throws_code',
       'inning_topbot_code', 'if_fielding_alignment_code',
       'of_fielding_alignment_code']

target = 'is_hr'

train_data = data.loc[data['test']==0, id_feat+feats+[target]]
test_data = data.loc[data['test']==1, id_feat+feats+[target]]
xtrain = xgb.DMatrix(train_data.loc[:, feats], train_data[target])
xtest = xgb.DMatrix(test_data.loc[:, feats], test_data[target])

kf = KFold(5)
s1 = pd.Series()
s2 = pd.Series()
for train_idx, test_idx in kf.split(train_data):
  _train_data = train_data.iloc[train_idx]
  _test_data = train_data.iloc[test_idx]
  _xtrain = xgb.DMatrix(_train_data.loc[:, feats], _train_data[target])
  _xtest = xgb.DMatrix(_test_data.loc[:, feats], _test_data[target])

  p = {
      'objective':'binary:logistic'
  }
  model = xgb.train(p, _xtrain)
  _s = pd.Series(model.predict(_xtest), index=_test_data.index)
  s1 = s1.append(_s)

  model = cb.CatBoostClassifier(iterations=250,verbose=False)
  model.fit(_train_data.loc[:, feats], _train_data[target])
  _s = pd.DataFrame(model.predict_proba(_test_data.loc[:, feats]), index=_test_data.index)[1]
  s2 = s2.append(_s)

train_data['xgb'] = s1
train_data['cb'] = s2

model = MLPClassifier()
model.fit(train_data.loc[:, ['xgb', 'cb']], train_data[target])

p = {
    'objective':'binary:logistic'
}
xgbm = xgb.train(p, xtrain)
test_data['xgb'] = pd.Series(xgbm.predict(xtest), index=test_data.index)

cbm = cb.CatBoostClassifier(iterations=250,verbose=False)
cbm.fit(train_data.loc[:, feats], train_data[target])
test_data['cb'] = pd.DataFrame(cbm.predict_proba(test_data.loc[:, feats]), index=test_data.index)[1]

test_data[f"{target}_pred"] = pd.DataFrame(model.predict_proba(test_data.loc[:, ['xgb', 'cb']]), index=test_data.index)[1]
test_data['uid'] = test_data['uid'].astype(str)
test_data.loc[:, ['uid', 'is_hr_pred']].rename(columns={'is_hr_pred':'is_hr'}).to_csv('drive/My Drive/community-comps/xhr/sample_solution.csv', index=False)

<ipython-input-146-85331fe8807a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s1 = pd.Series()
<ipython-input-146-85331fe8807a>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s2 = pd.Series()


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 11.2 MB/s eta 0:00:00


In [ ]:
import shap

In [ ]:
train_data.loc[:, ['xgb','cb']].sample(20000).shape

(20000, 2)

In [ ]:
x = train_data.loc[:, feats].sample(20000)
explainer = shap.KernelExplainer(cbm.predict_proba, x)
shap_values = explainer.shap_values(test_data.loc[:, feats])


  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
test_data.loc[test_data[target]==0, f"{target}_pred"].describe()

count    19249.000000
mean         0.007309
std          0.054748
min          0.000373
25%          0.000385
50%          0.000385
75%          0.000385
max          0.943571
Name: is_hr_pred, dtype: float64

In [ ]:
accuracy_score(test_data[target], test_data[f"{target}_pred"].round())

0.99075